# **ETLPipeline**

## Objectives

* Load healthcare insurance dataset from Kaggle (CSV format).
* Perform data cleaning: check nulls, handle duplicates, encode categorical features, and create BMI categories.
* Save the cleaned dataset for use in analysis and modeling notebooks.

## Inputs

* `insurance.csv` — the raw dataset downloaded from Kaggle, located in the `data/` folder.

## Outputs

* `cleaned_insurance.csv` — a cleaned and feature-engineered version of the dataset, saved in the `data/` folder.

## Additional Comments

* The BMI category feature is classified using WHO standard ranges: Underweight, Normal, Overweight, Obese.




### Data Cleaning

In [1]:
import numpy as np
import pandas as pd

In [3]:
# Load the dataset and cheking the first few rows
df = pd.read_csv('data/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


The dataset includes demographic and health-related information such as:

- Age
- Gender
- Body Mass Index (BMI)
- Number of children
- Smoking status
- Region (geographic)
- Insurance charges

In [4]:
# check the data types of the columns
print("\nData types:\n", df.dtypes)


Data types:
 age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object


In [5]:
# Check for missing values
print("Missing values:\n", df.isnull().sum())

Missing values:
 age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


No missing values detected across any columns.

In [6]:
# check for duplicates and print the count
duplicates = df.duplicated().sum()
print("Duplicate rows:", duplicates)

Duplicate rows: 1


In [7]:
# droping duplicate row
if duplicates > 0:
    df = df.drop_duplicates()
    print("Duplicates dropped. New shape:", df.shape)

Duplicates dropped. New shape: (1337, 7)


During the data cleaning process, one duplicate row was identified and removed. This ensures the dataset contains only unique records.

In [8]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


### Encode categorical variables

In [12]:
# Encoding categorical variables
df_encoded = pd.get_dummies(df, columns=['sex', 'smoker', 'region'])
df_encoded


,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,True,False,False,True,False,False,False,True
1,18,33.770,1,1725.55230,False,True,True,False,False,False,True,False
2,28,33.000,3,4449.46200,False,True,True,False,False,False,True,False
3,33,22.705,0,21984.47061,False,True,True,False,False,True,False,False
4,32,28.880,0,3866.85520,False,True,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,False,True,True,False,False,True,False,False
1334,18,31.920,0,2205.98080,True,False,True,False,True,False,False,False
1335,18,36.850,0,1629.83350,True,False,True,False,False,False,True,False
1336,21,25.800,0,2007.94500,True,False,True,False,False,False,False,True


In [13]:
# creating BMI categories
def categorize_bmi(bmi):
    if bmi < 18.5:
        return 'Under weight'
    elif 18.5 <= bmi < 24.9:
        return 'Normal weight'
    elif 25 <= bmi < 29.9:
        return 'Over weight'
    else:
        return 'Obesity'
    
df['bmi_category'] = df['bmi'].apply(categorize_bmi)
df

,age,sex,bmi,children,smoker,region,charges,bmi_category
0,19,female,27.900,0,yes,southwest,16884.92400,Over weight
1,18,male,33.770,1,no,southeast,1725.55230,Obesity
2,28,male,33.000,3,no,southeast,4449.46200,Obesity
3,33,male,22.705,0,no,northwest,21984.47061,Normal weight
4,32,male,28.880,0,no,northwest,3866.85520,Over weight
...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830,Obesity
1334,18,female,31.920,0,no,northeast,2205.98080,Obesity
1335,18,female,36.850,0,no,southeast,1629.83350,Obesity
1336,21,female,25.800,0,no,southwest,2007.94500,Over weight


In [14]:
# Displaying the count of each BMI category
print(df['bmi_category'].value_counts())

bmi_category
Obesity          721
Over weight      374
Normal weight    222
Under weight      20
Name: count, dtype: int64


In [16]:
# checking the correlation between numerical features
df1 = df_encoded.copy()
df1_correlation = df1.corr()
df1_correlation

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
age,1.000000,0.109344,0.041536,0.298308,0.019814,-0.019814,0.025587,-0.025587,0.001868,0.001495,-0.012311,0.009415
bmi,0.109344,1.000000,0.012755,0.198401,-0.046397,0.046397,-0.003746,0.003746,-0.138178,-0.136138,0.270057,-0.006211
children,0.041536,0.012755,1.000000,0.067389,-0.017848,0.017848,-0.007331,0.007331,-0.023202,0.026044,-0.023492,0.021538
charges,0.298308,0.198401,0.067389,1.000000,-0.058044,0.058044,-0.787234,0.787234,0.005945,-0.038695,0.073578,-0.043637
sex_female,0.019814,-0.046397,-0.017848,-0.058044,1.000000,-1.000000,0.076596,-0.076596,0.002008,0.012482,-0.017578,0.003767
sex_male,-0.019814,0.046397,0.017848,0.058044,-1.000000,1.000000,-0.076596,0.076596,-0.002008,-0.012482,0.017578,-0.003767
smoker_no,0.025587,-0.003746,-0.007331,-0.787234,0.076596,-0.076596,1.000000,-1.000000,-0.002597,0.036321,-0.068282,0.037168
smoker_yes,-0.025587,0.003746,0.007331,0.787234,-0.076596,0.076596,-1.000000,1.000000,0.002597,-0.036321,0.068282,-0.037168
region_northeast,0.001868,-0.138178,-0.023202,0.005945,0.002008,-0.002008,-0.002597,0.002597,1.000000,-0.319842,-0.345909,-0.320493
region_northwest,0.001495,-0.136138,0.026044,-0.038695,0.012482,-0.012482,0.036321,-0.036321,-0.319842,1.000000,-0.345909,-0.320493


### Key Observation
- Smoking status has the strongest correlation with charges (**positive correlation of 0.787** for smokers).
- Age and BMI are moderately correlated with charges, indicating older and heavier individuals tend to pay more.
- Region, sex, and number of children show minimal correlation with charges.

### Saving cleaned data set to new csv file

In [ ]:
cleaned_data = 'data/cleaned_insurance.csv'
df.to_csv(cleaned_data, index=False)
print(f'Cleaned data saved to {cleaned_data}')

Cleaned data saved to data/cleaned_insurance.csv
